In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import os
import json

In [ ]:
# Save Every Pages
savePath = ""
def save_html_per_page(source):
    
    
    with open(savePath +"/"+ str(get_index_for_save_page()) + ".html", "w") as file:       
        file.write(str(source))

#get index name for saving html source        
def get_index_for_save_page():
    index = -1
    arr = os.listdir(savePath)
    for x in arr:
        if(int(os.path.splitext(x)[0])> index):
            index = int(os.path.splitext(x)[0])              
    return index + 1    
    
    

In [24]:
#extract data from card. Card mean= car,house,ebay product list of card
def extract_data(card):
    try:
        my_json = json.loads(card.find("script").text)
    except Exception:
        return None
    # link
    try:
        link = re.sub('\s+', '', card.find("div", {"class": "list-card-info"}).find("a")['href'])
    except Exception:
        link = ""

    # name
    try:
        name = re.sub('\s+', '', my_json["name"])
    except Exception:
        name = ""

    # type
    try:
        type = re.sub('\s+', '', my_json["@type"])
    except Exception:
        type = ""

    # latitude
    try:
        latitude =  my_json["geo"]["latitude"]
    except Exception:
        latitude = ""

    # longitude
    try:
        longitude =  my_json["geo"]["longitude"]
    except Exception:
        longitude = ""

    # price
    try:
        price = re.sub('\s+', '', card.find("div", {"class": "list-card-price"}).text)
    except Exception:
        price = ""

    # bds
    try:
        bds = re.sub('\s+', '', card.find_all("abbr", {"class": "list-card-label"})[0].parent.text)
    except Exception:
        bds = ""

    # ba
    try:
        ba = re.sub('\s+', '', card.find_all("abbr", {"class": "list-card-label"})[1].parent.text)
    except Exception:
        ba = ""

    # sqft
    try:
        sqft = re.sub('\s+', '', card.find_all("abbr", {"class": "list-card-label"})[2].parent.text)
    except Exception:
        sqft = ""

    # status
    try:
        status = re.sub('\s+', '', card.find("li", {"class": "list-card-statusText"}).text)
    except Exception:
        status = ""

    data = [name, type, price, bds, ba, sqft, status, latitude, longitude, link]
    
    return data


In [25]:

#header need for zillow trap human check automation
yourCookie=""
headers = {
          'cookie':yourCookie,
          'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
          }

data = []
url_start = 'https://www.zillow.com/nm/rentals/' 
url_end='_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A12%7D%2C%22mapBounds%22%3A%7B%22west%22%3A-140.567083125%2C%22east%22%3A-71.48505187500002%2C%22south%22%3A24.06379380943862%2C%22north%22%3A43.275025020418475%7D%2C%22mapZoom%22%3A4%2C%22usersSearchTerm%22%3A%22New%20Mexico%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A41%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D'

#You can select page count such as 10,15,100 your choice
for i in range(0,13):
    response = requests.get(url_start + str(i) + url_end ,  headers=headers)
    
    source = BeautifulSoup(response.content, 'html.parser')
    
    save_html_per_page(response.content)
    
    cards = source.find("div",{"class","result-list-container"}).find("ul",{"class":"photo-cards photo-cards_wow photo-cards_short"}).find_all("li",{"class":""})

    for card in cards:
        value = extract_data(card)
        #if value if null, undefined card information. This is may not correct data
        if(value != None):
            data.append(value)

               

In [26]:
#Save 
df = pd.DataFrame(data, columns = ['Name', 'Type', 'Price', 'bds', 'ba', 'sqft', 'status', 'latitude', 'longitude', 'link'])
df.to_excel ('C:/Users/nurul/OneDrive/Masaüstü/New folder/export_dataframe.xlsx', index = False, header=True)